In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
flight_data = pd.read_csv("C:/Users/Emily Shao/Desktop/archive/itineraries.csv")

encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(flight_data[['startingAirport', 'endingAirport', 'flightDate']])

X = pd.concat([pd.DataFrame(encoded_features), flight_data[['price']]], axis=1)
y = X.pop('price')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)